In [1]:
import tensorflow as tf
import numpy as np

In [2]:
Blinks = np.load('./npy/Blinks_F3.npy')
Labels = np.load('./npy/Labels_F3.npy')
BlinksTest = np.load('./npy/BlinksTest_F3.npy')
LabelsTest = np.load('./npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [4]:
def create_model(feature_size, post_fc2_size_per_layer=16, step_size=30):
    #Feature Transformation Layers
    def clean_split_and_squeeze(x, step_size):
        trash,output_RNN=tf.split(x,[step_size-1,1],axis=0,name='layers')  #size of output_RNN[lay] is (step,batch,hsize),
        return tf.squeeze(output_RNN,axis=0)

    input = tf.keras.Input(shape=(None, None, feature_size))
    #   F = ReLU(BW + b)
    F = tf.keras.layers.Dense(activation='relu', units=32)(tf.reshape(input, [-1, feature_size]))
    F = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(tf.reshape(F, [-1,step_size,32]))

    #SCATOLA NERA - LSTM (prima o poi la faremo)
    #lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
    #rnn_cells = [tf.keras.layers.LSTMCell(4) for _ in range(29)]
    #stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
    #lstm_layer = tf.keras.layers.RNN(stacked_lstm, return_sequences=True, return_state=True)
    #output_RNN_set = lstm_layer(F)
    '''
    # Fully Connected Layer 1
    f1, f2, f3, f4 = clean_split_and_squeeze(output_RNN_set[0], step_size),\
                     clean_split_and_squeeze(output_RNN_set[1], step_size),\
                     clean_split_and_squeeze(output_RNN_set[2], step_size),\
                     clean_split_and_squeeze(output_RNN_set[3], step_size)
    f1post = tf.keras.layers.Dense(units=16)(f1)
    f2post = tf.keras.layers.Dense(units=16)(f2)
    f3post = tf.keras.layers.Dense(units=16)(f3)
    f4post = tf.keras.layers.Dense(units=16)(f4)
    post = tf.concat([f1post,f2post,f3post,f4post], axis=1)
    FC1 = tf.keras.layers.Dense(activation='relu', units=16)(post)
    FC1 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC1)

    #Fully Connected Layer 2

    FC2 = tf.keras.layers.Dense(activation='relu', units=16)(FC1)
    FC2 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC2)

    #Fully Connected Layer 3

    FC3 = tf.keras.layers.Dense(activation='relu', units=16)(FC2)
    FC3 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC3)

    #Fully Connected Layer 4

    FC4 = tf.keras.layers.Dense(activation='relu', units=8)(FC3)
    FC4 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC4)

    #Output Layer

    output = tf.keras.layers.Dense(units=1)(FC4)
    '''
    model = tf.keras.models.Model(inputs=[input], outputs=[F])
    return model

In [6]:
model = create_model(feature_size=4)
model.compile()

In [37]:
def test_model():
    input = tf.keras.Input(shape=(30, 4))
    x = tf.keras.layers.Masking(mask_value=0.0)(input)
    x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    # Output layer
    y = tf.keras.layers.Dense(1, activation='tanh')(x)
    model = tf.keras.models.Model(inputs=[input], outputs=[y])
    return model
model2 = test_model()
model2.summary()
loss=tf.keras.losses.binary_crossentropy
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0053)
model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 30, 4)]           0         
_________________________________________________________________
masking_10 (Masking)         (None, 30, 4)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                4736      
_________________________________________________________________
dense_31 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 17        
Total params: 5,281
Trainable params: 5,281
Non-trainable params: 0
___________________________________________________

In [39]:
Labels2 = Labels
LabelsTest[LabelsTest==10] = 1
Labels2[Labels2==10] = 1

In [41]:
model2.fit(x= Blinks, y = Labels2, validation_data=(BlinksTest, LabelsTest), epochs=100, batch_size=32)

Epoch 1/100
38/38 [==============================] - 7s 178ms/step - loss: 0.1101 - accuracy: 0.9652 - val_loss: 2.2081 - val_accuracy: 0.6427